# ORIE 5380 Final Project
## by Chenyang Yan(cy477)

In [1]:
from ortools.linear_solver import pywraplp
import numpy as np
import pandas as pd
import math

# 1. Formulation of the problem

The problem ask us to meet the requirements of the dose and since the radiation can hurt body, we should use less beamlets as possible. Problem provides us with the dose matrix which gives the dose delivered to each voxel from each beamlet when one unit of that beamlet is used. Beamlets are from 1 to 60. Voxels are from 1 to 400. I then use the index to remember the position of different regions as below.

In [2]:
DoseMatrix = pd.read_excel("DoseMatrix.xlsx")

In [3]:
# Based on the VisualDemo, list the postion of each region
right_femur = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,
               182,183,184,185,202,203,204,205,222,223,224,225,243,244]
# Left femur has the same size as right femur but move rightward for 13 voxels
left_femur = [i + 14 for i in right_femur]
bladder = [87,88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,
           127,128,129,130,131,132,133,134,135,147,148,149,150,151,152,153,154]
# CTV has the same size as bladder but just moving 4 lines down
ctv = [i + 80 for i in bladder]
rectal_solid = [248,249,250,251,252,253,268,269,270,271,272,273,274,
               288,289,290,291,292,293,294,308,309,310,311,312,313]
# All other points are in unspecified
unspecified = []
for i in range(1,401):
    if right_femur.count(i) == 0 and left_femur.count(i) == 0 and ctv.count(i) == 0 and rectal_solid.count(i) == 0:
        unspecified.append(i)

# Constrains for all region:

## For the CTV:
#### Receive a uniform dose of 82.8.
#### With 5% accuracy and consideration of cold and high effect, the range is [79.1, 86.5].

## For the Bladder:
#### Max dose is 81.
#### Average dose should be <= 50.
#### At most 10% of bladder should receive > 65.

## For the Rectum:
#### Max dose is 79.2.
#### Average dose should be <= 40.

## For the Unspecified:
#### Max dose is 72.

## For the Left femur head:
#### Max dose is 50.
#### At most 15% of Left femur head should receive > 40.

## For the Right femur head:
#### Max dose is 50.
#### At most 15% of Left Right head should receive > 40.

In [4]:
# Don't need the first column because it's the useless index.
# Now each column of the dose represents the unit intensity of a beamlet on 400 voxels.
dose = DoseMatrix.iloc[:,1:].to_numpy()
dose[:,0].shape

(400,)

### Start experiment

In [5]:
solver = pywraplp.Solver('Final project',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
objective=solver.Objective()
objective.SetMinimization()
# Set the variable to control the unit density of each beamlet
beam_value = [0 for i in range(60)]
# Initialize the variables
for i in range(60):
    beam_value[i] = solver.IntVar(0, solver.infinity(), str(i))


In [6]:
# Add the constraints
# CTV
for index in ctv:
    solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) >= 79.1)
    solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) <= 86.5)
# Unspecified
for index in unspecified:
    solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 72)
# Set the objective
solver.Minimize(solver.Sum(beam_value[i] for i in range(60)))

In [7]:
#run the solver
status = solver.Solve()
#find the optimal solution by solver
if status == solver.OPTIMAL:
    print("Solver reaches the optimal solution with objective function = " + str(objective.Value()))
    beams = []
    for i in range(60):
        beams.append(beam_value[i].solution_value())

else:
    print('Solver cannot find optimal solution')
# A feasible solution
beams = np.floor(beams)
for i in range(6):
    for j in range(10):
        print("Number " + str(i * 10 + j + 1) + " beam has " + str(beams[i * 10 + j]) + " strength")
    print("==========================")

Solver reaches the optimal solution with objective function = 464.99185271177623
Number 1 beam has 14.0 strength
Number 2 beam has 19.0 strength
Number 3 beam has 0.0 strength
Number 4 beam has 0.0 strength
Number 5 beam has 7.0 strength
Number 6 beam has 2.0 strength
Number 7 beam has 0.0 strength
Number 8 beam has 0.0 strength
Number 9 beam has 0.0 strength
Number 10 beam has 0.0 strength
Number 11 beam has 0.0 strength
Number 12 beam has 0.0 strength
Number 13 beam has 0.0 strength
Number 14 beam has 0.0 strength
Number 15 beam has 13.0 strength
Number 16 beam has 17.0 strength
Number 17 beam has 0.0 strength
Number 18 beam has 46.0 strength
Number 19 beam has 29.0 strength
Number 20 beam has 0.0 strength
Number 21 beam has 0.0 strength
Number 22 beam has 25.0 strength
Number 23 beam has 47.0 strength
Number 24 beam has 0.0 strength
Number 25 beam has 32.0 strength
Number 26 beam has 0.0 strength
Number 27 beam has 0.0 strength
Number 28 beam has 0.0 strength
Number 29 beam has 0.0 

In [8]:
def get_precent(beams, dose, bladder, left_femur, right_femur):
    fre_b = []
    fre_l = []
    fre_r = []
    for index in bladder:
        if sum(beams[i] * dose[index - 1,i] for i in range(60)) > 65:
            fre_b.append(1)
        else:
            fre_b.append(0)
    print("The percentage of doses in bladder that over 65 is " + str(sum(fre_b)/len(fre_b)))
    for index in left_femur:
        if sum(beams[i] * dose[index - 1,i] for i in range(60)) > 40:
            fre_l.append(1)
        else:
            fre_l.append(0)
    print("The percentage of doses in left femur that over 40 is " + str(sum(fre_l)/len(fre_l)))
    for index in right_femur:
        if sum(beams[i] * dose[index - 1,i] for i in range(60)) > 40:
            fre_r.append(1)
        else:
            fre_r.append(0)
    print("The percentage of doses in right femur that over 40 is " + str(sum(fre_r)/len(fre_r)))

In [9]:
get_precent(beams, dose, bladder, left_femur, right_femur)

The percentage of doses in bladder that over 65 is 0.23529411764705882
The percentage of doses in left femur that over 40 is 0.6785714285714286
The percentage of doses in right femur that over 40 is 0.42857142857142855


### Further experiment A

In [10]:
solver = pywraplp.Solver('Final project',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
objective=solver.Objective()
objective.SetMinimization()
# Set the variable to control the unit density of each beamlet
beam_value = [0 for i in range(60)]
# Initialize the variables
for i in range(60):
    beam_value[i] = solver.IntVar(0, solver.infinity(), str(i))

In [11]:
# Add the constraints
# CTV
for index in ctv:
    solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) >= 79.1)
    solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) <= 86.5)

# Bladder
for index in bladder:
    solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 81)
solver.Add(solver.Sum(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60))for index in bladder) / len(bladder) <= 60)

# Rectum
for index in rectal_solid:
    solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 79.2)
solver.Add(solver.Sum(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60))for index in rectal_solid) / len(rectal_solid) <= 55)

# Unspecified
for index in unspecified:
    solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 75)

# Left femur head
for index in left_femur:
    solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 60)

# Right femur head
for index in right_femur:
    solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 55)

# Set the objective
solver.Minimize(solver.Sum(beam_value[i] for i in range(60)))

In [12]:
#run the solver
status = solver.Solve()
#find the optimal solution by solver
if status == solver.OPTIMAL:
    print("Solver reaches the optimal solution with objective function = " + str(objective.Value()))
    beams = []
    for i in range(60):
        beams.append(beam_value[i].solution_value())

else:
    print('Solver cannot find optimal solution')
# A feasible solution
beams = np.floor(beams)
for i in range(6):
    for j in range(10):
        print("Number " + str(i * 10 + j + 1) + " beam has " + str(beams[i * 10 + j]) + " strength")
    print("==========================")

Solver reaches the optimal solution with objective function = 451.27775018973307
Number 1 beam has 0.0 strength
Number 2 beam has 26.0 strength
Number 3 beam has 0.0 strength
Number 4 beam has 0.0 strength
Number 5 beam has 7.0 strength
Number 6 beam has 0.0 strength
Number 7 beam has 0.0 strength
Number 8 beam has 0.0 strength
Number 9 beam has 0.0 strength
Number 10 beam has 23.0 strength
Number 11 beam has 0.0 strength
Number 12 beam has 0.0 strength
Number 13 beam has 0.0 strength
Number 14 beam has 0.0 strength
Number 15 beam has 0.0 strength
Number 16 beam has 26.0 strength
Number 17 beam has 0.0 strength
Number 18 beam has 52.0 strength
Number 19 beam has 10.0 strength
Number 20 beam has 0.0 strength
Number 21 beam has 0.0 strength
Number 22 beam has 9.0 strength
Number 23 beam has 55.0 strength
Number 24 beam has 0.0 strength
Number 25 beam has 28.0 strength
Number 26 beam has 0.0 strength
Number 27 beam has 0.0 strength
Number 28 beam has 0.0 strength
Number 29 beam has 0.0 st

In [13]:
get_precent(beams, dose, bladder, left_femur, right_femur)

The percentage of doses in bladder that over 65 is 0.2647058823529412
The percentage of doses in left femur that over 40 is 0.5
The percentage of doses in right femur that over 40 is 0.25


### Further experiment A imporved

In [14]:
solver = pywraplp.Solver('Final project',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
objective=solver.Objective()
objective.SetMinimization()
# Set the variable to control the unit density of each beamlet
beam_value = [0 for i in range(60)]
# Initialize the variables
for i in range(60):
    beam_value[i] = solver.IntVar(0, solver.infinity(), str(i))
right_binary = [0 for i in range(len(right_femur))]
left_binary = [0 for i in range(len(left_femur))]
for i in range(len(right_femur)):
    right_binary[i] = solver.IntVar(0, 1, "right " + str(i))
    left_binary[i] = solver.IntVar(0, 1, "left " + str(i))

In [15]:
# Add the constraints
# CTV
for index in ctv:
    solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) >= 79.1)
    solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) <= 86.5)

# # Bladder
# for index in bladder:
#     solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 81)
# solver.Add(solver.Sum(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60))for index in bladder) / len(bladder) <= 60)

# # Rectum
# for index in rectal_solid:
#     solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 79.2)
# solver.Add(solver.Sum(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60))for index in rectal_solid) / len(rectal_solid) <= 55)

# # Unspecified
# for index in unspecified:
#     solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 75)

# Left femur head
for index in range(len(left_femur)):
    solver.Add(solver.Sum((beam_value[i] * dose[left_femur[index] - 1,i] for i in range(60))) <= 60)
for index in range(len(left_femur)):
    solver.Add(solver.Sum((beam_value[i] * dose[left_femur[index] - 1,i] for i in range(60))) 
               <= 40 + 10000 * left_binary[index])
solver.Add(solver.Sum(left_binary) <= 4)
# Right femur head
for index in range(len(right_femur)):
    solver.Add(solver.Sum((beam_value[i] * dose[right_femur[index] - 1,i] for i in range(60))) <= 55)
for index in range(len(right_femur)):
    solver.Add(solver.Sum((beam_value[i] * dose[right_femur[index] - 1,i] for i in range(60))) 
               <= 40 + 10000 * right_binary[index])
solver.Add(solver.Sum(right_binary) <= 4)

# Set the objective
solver.Minimize(solver.Sum(beam_value[i] for i in range(60)))

In [16]:
#run the solver
status = solver.Solve()
#find the optimal solution by solver
if status == solver.OPTIMAL:
    print("Solver reaches the optimal solution with objective function = " + str(objective.Value()))
    beams = []
    for i in range(60):
        beams.append(beam_value[i].solution_value())

else:
    print('Solver cannot find optimal solution')
# A feasible solution
beams = np.floor(beams)
for i in range(6):
    for j in range(10):
        print("Number " + str(i * 10 + j + 1) + " beam has " + str(beams[i * 10 + j]) + " strength")
    print("==========================")

Solver reaches the optimal solution with objective function = 446.0822859300855
Number 1 beam has 0.0 strength
Number 2 beam has 32.0 strength
Number 3 beam has 0.0 strength
Number 4 beam has 0.0 strength
Number 5 beam has 8.0 strength
Number 6 beam has 7.0 strength
Number 7 beam has 0.0 strength
Number 8 beam has 0.0 strength
Number 9 beam has 5.0 strength
Number 10 beam has 8.0 strength
Number 11 beam has 0.0 strength
Number 12 beam has 0.0 strength
Number 13 beam has 0.0 strength
Number 14 beam has 0.0 strength
Number 15 beam has 0.0 strength
Number 16 beam has 30.0 strength
Number 17 beam has 0.0 strength
Number 18 beam has 52.0 strength
Number 19 beam has 13.0 strength
Number 20 beam has 0.0 strength
Number 21 beam has 0.0 strength
Number 22 beam has 13.0 strength
Number 23 beam has 52.0 strength
Number 24 beam has 0.0 strength
Number 25 beam has 30.0 strength
Number 26 beam has 0.0 strength
Number 27 beam has 0.0 strength
Number 28 beam has 0.0 strength
Number 29 beam has 0.0 str

In [17]:
get_precent(beams, dose, bladder, left_femur, right_femur)

The percentage of doses in bladder that over 65 is 0.3235294117647059
The percentage of doses in left femur that over 40 is 0.5357142857142857
The percentage of doses in right femur that over 40 is 0.10714285714285714


### Alternative method solving percentage problem

In [92]:
def run_experiment(beam_list, obj_list, index_right, index_left):
    solver = pywraplp.Solver('Final project',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    objective=solver.Objective()
    objective.SetMinimization()
    # Set the variable to control the unit density of each beamlet
    beam_value = [0 for i in range(60)]
    # Initialize the variables
    for i in range(60):
        beam_value[i] = solver.IntVar(0, solver.infinity(), str(i))

    # Add the constraints
    # CTV
    for index in ctv:
        solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) >= 79.1)
        solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) <= 86.5)

#     # Bladder
#     for index in bladder:
#         solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 81)
#     solver.Add(solver.Sum(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60))for index in bladder) / len(bladder) <= 60)

#     # Rectum
#     for index in rectal_solid:
#         solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 79.2)
#     solver.Add(solver.Sum(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60))for index in rectal_solid) / len(rectal_solid) <= 55)

#     # Unspecified
#     for index in unspecified:
#         solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 75)

    # Left femur head
    for index in range(len(left_femur)):
        solver.Add(solver.Sum((beam_value[i] * dose[left_femur[index] - 1,i] for i in range(60))) <= 60)
    for index in index_left:
        solver.Add(solver.Sum((beam_value[i] * dose[left_femur[index] - 1,i] for i in range(60))) <= 40)
    
    # Right femur head
    for index in range(len(right_femur)):
        solver.Add(solver.Sum((beam_value[i] * dose[right_femur[index] - 1,i] for i in range(60))) <= 55)
    for index in index_right:
        solver.Add(solver.Sum((beam_value[i] * dose[right_femur[index] - 1,i] for i in range(60))) <= 40)
    
    # Set the objective
    solver.Minimize(solver.Sum(beam_value[i] for i in range(60)))
    
    #run the solver
    status = solver.Solve()
    #find the optimal solution by solver
    if status == solver.OPTIMAL:
        obj_list.append(objective.Value())
        beams = []
        for i in range(60):
            beams.append(np.floor(beam_value[i].solution_value()))
        beam_list.append(beams)
    return beam_list, obj_list

In [93]:
beam_list = []
obj_list = []
for i in range(100):
    length_right_femur = len(right_femur)
    length_left_femur = len(left_femur)
    index_right = []
    index_left = []
    for i in range(len(right_femur) - 4):
        index_right.append(np.random.randint(length_right_femur))
        index_left.append(np.random.randint(length_left_femur))
    beam_list, obj_list = run_experiment(beam_list, obj_list, index_right, index_left)
    

In [94]:
obj_list

[465.01757445791804, 465.01757445792185, 465.01757445790884, 454.8678244044606]

In [95]:
min_index = obj_list.index(min(obj_list))
for i in range(6):
    for j in range(10):
        print("Number " + str(i * 10 + j + 1) + " beam has " + str(beam_list[min_index][i * 10 + j]) + " strength")
    print("==========================")

Number 1 beam has 0.0 strength
Number 2 beam has 33.0 strength
Number 3 beam has 16.0 strength
Number 4 beam has 0.0 strength
Number 5 beam has 3.0 strength
Number 6 beam has 30.0 strength
Number 7 beam has 0.0 strength
Number 8 beam has 0.0 strength
Number 9 beam has 31.0 strength
Number 10 beam has 13.0 strength
Number 11 beam has 0.0 strength
Number 12 beam has 0.0 strength
Number 13 beam has 0.0 strength
Number 14 beam has 0.0 strength
Number 15 beam has 0.0 strength
Number 16 beam has 0.0 strength
Number 17 beam has 0.0 strength
Number 18 beam has 27.0 strength
Number 19 beam has 25.0 strength
Number 20 beam has 0.0 strength
Number 21 beam has 0.0 strength
Number 22 beam has 21.0 strength
Number 23 beam has 32.0 strength
Number 24 beam has 0.0 strength
Number 25 beam has 0.0 strength
Number 26 beam has 0.0 strength
Number 27 beam has 0.0 strength
Number 28 beam has 0.0 strength
Number 29 beam has 0.0 strength
Number 30 beam has 0.0 strength
Number 31 beam has 0.0 strength
Number 3

In [96]:
get_precent(beam_list[min_index], dose, bladder, left_femur, right_femur)

The percentage of doses in bladder that over 65 is 0.6764705882352942
The percentage of doses in left femur that over 40 is 0.17857142857142858
The percentage of doses in right femur that over 40 is 0.0


### Final experiment

In [235]:
def run_experiment_final(beam_list, obj_list, index_right, index_left, index_bladder):
    solver = pywraplp.Solver('Final project',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    objective=solver.Objective()
    objective.SetMinimization()
    # Set the variable to control the unit density of each beamlet
    beam_value = [0 for i in range(60)]
    # Initialize the variables
    for i in range(60):
        beam_value[i] = solver.IntVar(0, solver.infinity(), str(i))

    # Add the constraints
    # CTV
    for index in ctv:
        solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) >= 79.1)
        solver.Add(solver.Sum(beam_value[i] * dose[index - 1,i] for i in range(60)) <= 86.5)

#     # Bladder
    for index in bladder:
        solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 81)
    solver.Add(solver.Sum(solver.Sum(beam_value[i] * dose[index - 1,i] 
                                     for i in range(60))for index in bladder) / len(bladder) <= 65)
#     for index in index_bladder:
#         solver.Add(solver.Sum((beam_value[i] * dose[bladder[index] - 1,i] for i in range(60))) <= 65)

    # Rectum
    for index in rectal_solid:
        solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 79.2)
    solver.Add(solver.Sum(solver.Sum(beam_value[i] * dose[index - 1,i] 
                                     for i in range(60))for index in rectal_solid) / len(rectal_solid) <= 65)

#     # Unspecified
#     for index in unspecified:
#         solver.Add(solver.Sum((beam_value[i] * dose[index - 1,i] for i in range(60))) <= 80)

    # Left femur head
    for index in range(len(left_femur)):
        solver.Add(solver.Sum((beam_value[i] * dose[left_femur[index] - 1,i] for i in range(60))) <= 60)
    for index in index_left:
        solver.Add(solver.Sum((beam_value[i] * dose[left_femur[index] - 1,i] for i in range(60))) <= 40)
    
    # Right femur head
    for index in range(len(right_femur)):
        solver.Add(solver.Sum((beam_value[i] * dose[right_femur[index] - 1,i] for i in range(60))) <= 55)
    for index in index_right:
        solver.Add(solver.Sum((beam_value[i] * dose[right_femur[index] - 1,i] for i in range(60))) <= 40)
    
    # Set the objective
    solver.Minimize(solver.Sum(beam_value[i] for i in range(60)))
    
    #run the solver
    status = solver.Solve()
    #find the optimal solution by solver
    if status == solver.OPTIMAL:
        obj_list.append(objective.Value())
        beams = []
        for i in range(60):
            beams.append(np.floor(beam_value[i].solution_value()))
        beam_list.append(beams)
    return beam_list, obj_list

In [236]:
beam_list = []
obj_list = []
for i in range(100):
    length_right_femur = len(right_femur)
    length_left_femur = len(left_femur)
    length_bladder = len(bladder)
    index_right = []
    index_left = []
    index_bladder = []
    for i in range(len(right_femur) - 17):
        index_right.append(np.random.randint(length_right_femur))
        index_left.append(np.random.randint(length_left_femur))
    for i in range(len(bladder) - 20):
        index_bladder.append(np.random.randint(length_bladder))
    beam_list, obj_list = run_experiment_final(beam_list, obj_list, index_right, index_left, index_bladder)
    

In [237]:
obj_list

[450.4606115909367,
 451.7964108566752,
 454.88988454671403,
 456.2106041874918,
 451.7964108566751,
 449.4335986234195,
 451.7964108566751,
 449.7797933343167,
 449.91442922686633,
 450.26839363673344,
 452.74256674137644,
 455.2592259811026,
 456.24546340545476]

In [238]:
min_index = obj_list.index(min(obj_list))
for i in range(6):
    for j in range(10):
        print("Number " + str(i * 10 + j + 1) + " beam has " + str(beam_list[min_index][i * 10 + j]) + " strength")
    print("==========================")

Number 1 beam has 0.0 strength
Number 2 beam has 53.0 strength
Number 3 beam has 0.0 strength
Number 4 beam has 0.0 strength
Number 5 beam has 37.0 strength
Number 6 beam has 0.0 strength
Number 7 beam has 0.0 strength
Number 8 beam has 0.0 strength
Number 9 beam has 0.0 strength
Number 10 beam has 0.0 strength
Number 11 beam has 0.0 strength
Number 12 beam has 0.0 strength
Number 13 beam has 0.0 strength
Number 14 beam has 0.0 strength
Number 15 beam has 0.0 strength
Number 16 beam has 26.0 strength
Number 17 beam has 0.0 strength
Number 18 beam has 47.0 strength
Number 19 beam has 16.0 strength
Number 20 beam has 0.0 strength
Number 21 beam has 0.0 strength
Number 22 beam has 1.0 strength
Number 23 beam has 46.0 strength
Number 24 beam has 0.0 strength
Number 25 beam has 0.0 strength
Number 26 beam has 0.0 strength
Number 27 beam has 0.0 strength
Number 28 beam has 0.0 strength
Number 29 beam has 0.0 strength
Number 30 beam has 0.0 strength
Number 31 beam has 32.0 strength
Number 32 

In [239]:
get_precent(beam_list[min_index], dose, bladder, left_femur, right_femur)

The percentage of doses in bladder that over 65 is 0.3235294117647059
The percentage of doses in left femur that over 40 is 0.32142857142857145
The percentage of doses in right femur that over 40 is 0.10714285714285714
